In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json


In [69]:

path = 'E:\Technologie\Master\Webm\wem'

with open(f'{path}\\movies.json',encoding="utf8") as f:
    data = json.load(f)

with open(f'{path}\\nlp.json',encoding="utf8") as f:
    data_nlp = json.load(f)
 


In [ ]:
df_nlp = pd.DataFrame(data_nlp).T
df_data = pd.DataFrame(data)

In [ ]:
# set the uuid as index
df_data.set_index('uuid', inplace=True)

In [ ]:
# join the two dataframes on the index and uuid
df = df_data.join(df_nlp, how="inner")

In [ ]:
# add a column uuid from the index
df['uuid'] = df.index

In [ ]:
def loadJson(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data

def moving_average(x, width):
    return np.convolve(x, np.ones(width), 'valid') / width

def computeNormAvg(movie, avgWidth):
    neg = [entry[1] for entry in movie]
    pos = [entry[2] for entry in movie]
    avgNeg = moving_average(neg, avgWidth)
    avgPos = moving_average(pos, avgWidth)
    diff = []
    for i in range(len(avgPos)):
        diff.append(avgPos[i] - avgNeg[i])
    x = [entry[0] for entry in movie[:len(avgPos)]]
    factor = 100 / x[-1]
    normx = [r * factor for r in x]
    return normx, avgNeg, avgPos, diff

def plot(x, neg=None, pos=None, diff=None, poly=None):
    plt.figure(figsize=(8, 4), layout='constrained')
    if neg is not None: plt.plot(x, neg, color="red", label="neg")
    if pos is not None: plt.plot(x, pos, color="green", label="pos")
    if diff is not None: plt.plot(x, diff, color="blue", label="delta")
    if poly is not None: plt.plot(x, poly, color="green", label="fit")
    plt.axhline(y=0, color="black", linestyle='--')
    plt.xlabel("time [s]")
    plt.ylabel("sentiment")
    plt.legend()

In [60]:
# get the first row of the dataframe
rows = df.iloc[:2]


In [70]:
for l,s in rows.iterrows():
    n_path = f'{path}\\analysis\{s["uuid"]}.json'

In [71]:
analysis_data = loadJson(n_path)

In [ ]:
x, neg, pos, diff = computeNormAvg(analysis_data, 128)
fit1 = list(np.polyfit(x, diff, 16))
fit1.reverse()
func = [sum([val**i * fit1[i] for i in range(len(fit1))]) for val in x]
plot(x, diff=diff, poly=func)